# 🧪 Experiment Template

این نوتبوک نشون میده چطور از سیستم MLOps استفاده کنید.

## Workflow

```
1. Load data
2. Feature engineering (your changes)
3. Train with tracking
4. Compare results
5. Promote best model
```

## Setup

In [1]:
print('Hello')

Hello


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# Import our MLOps tracker
import sys
sys.path.insert(0, '../..')
from mlops import ExperimentTracker

## 1. Load Data

In [ ]:
# Load your feature data
df = pd.read_csv('../../data/user_features.csv')

print(f"Samples: {len(df):,}")
print(f"Features: {df.shape[1] - 2}")  # minus user_id and is_churned
print(f"Churn rate: {df['is_churned'].mean():.1%}")

## 2. Prepare Data

In [ ]:
# Feature columns (exclude user_id and target)
feature_cols = [c for c in df.columns if c not in ['user_id', 'is_churned']]

X = df[feature_cols]
y = df['is_churned']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train):,}")
print(f"Test: {len(X_test):,}")

## 3. Train with Experiment Tracking

### Example: Baseline Model

In [ ]:
# Initialize tracker
tracker = ExperimentTracker()

In [ ]:
# Run an experiment
with tracker.run("baseline_xgboost"):
    
    # Define model
    model = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    
    # Log parameters
    tracker.log_params({
        "n_estimators": 100,
        "max_depth": 6,
        "learning_rate": 0.1,
    })
    
    # Train
    model.fit(X_train, y_train)
    
    # Log metrics (automatically calculates ROC-AUC, F1, etc.)
    metrics = tracker.log_metrics(
        model, X_test, y_test,
        X_train=X_train, y_train=y_train
    )
    
    # Save model
    tracker.log_model(model, feature_names=feature_cols)

### Example: Experiment with Different Parameters

In [ ]:
with tracker.run("xgb_deeper_trees"):
    
    model = XGBClassifier(
        n_estimators=200,
        max_depth=10,  # deeper
        learning_rate=0.05,  # slower learning
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    
    tracker.log_params({
        "n_estimators": 200,
        "max_depth": 10,
        "learning_rate": 0.05,
    })
    
    model.fit(X_train, y_train)
    tracker.log_metrics(model, X_test, y_test)
    tracker.log_model(model, feature_names=feature_cols)

## 4. Compare Experiments

In [ ]:
# Compare all experiments
tracker.compare()

In [ ]:
# Compare by different metric
tracker.compare(metric="f1")

In [ ]:
# Compare by inference speed
tracker.compare(metric="inference_time_ms")

## 5. Promote Best Model to Production

این دستور مدل رو کپی میکنه به `app/model.pkl` که FastAPI ازش استفاده میکنه.

In [ ]:
# Promote a specific experiment
tracker.promote("baseline_xgboost")

In [ ]:
# Or promote the best one automatically
from mlops import promote_best

promote_best(metric="roc_auc")

## 6. View in MLflow UI

برای دیدن همه آزمایش‌ها با UI گرافیکی:

```bash
mlflow ui --port 5000 --backend-store-uri ./mlruns
```

بعد برو به: http://localhost:5000

---

## 📝 Template: Your New Experiment

از این template برای آزمایش‌های جدید استفاده کن:

In [ ]:
# === YOUR EXPERIMENT ===

EXPERIMENT_NAME = "describe_what_you_changed"  # e.g., "added_sentiment_scores"

with tracker.run(EXPERIMENT_NAME):
    
    # --- Your feature engineering changes ---
    # X_train_new = add_new_features(X_train)
    # X_test_new = add_new_features(X_test)
    
    # --- Model definition ---
    model = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    
    # --- Log parameters ---
    tracker.log_params({
        "n_estimators": 100,
        "max_depth": 6,
        "note": "describe your changes",
    })
    
    # --- Train ---
    model.fit(X_train, y_train)
    
    # --- Evaluate & Log ---
    tracker.log_metrics(model, X_test, y_test)
    tracker.log_model(model, feature_names=feature_cols)

# --- Compare with previous experiments ---
tracker.compare()